In [43]:
import numpy as np
from numba import jit
from sympy import sieve, mod_inverse

In [47]:
def build_totient_table(max_num):
    return list(sieve.totientrange(1, max_num + 1))

def build_fact_n_div_n_pow_n_mod_m_table(max_num, modulus):
    results = [1] * (max_num + 1)
    factorials = results[:]

    for num in range(2, len(results)):
        factorials[num] = (factorials[num - 1] * num) % modulus
        results[num] = (factorials[num] * mod_inverse(pow(num, num, modulus), modulus)) % modulus

    return results

@jit
def build_mobius_table(max_num):
    mu = np.ones(max_num + 1, dtype=np.int64)
    mu[0] = 0

    for i in range(2, int(np.sqrt(max_num)) + 1):
        if mu[i] == 1:
            for j in range(1, (max_num // i) + 1):
                mu[i * j] *= -i
            for j in range(1, (max_num // (i ** 2)) + 1):
                mu[i * i * j] = 0

    for i in range(2, max_num + 1):
        if mu[i] == i:
            mu[i] = 1
        elif mu[i] == -i:
            mu[i] = -1
        elif mu[i] < 0:
            mu[i] = 1
        elif mu[i] > 0:
            mu[i] = -1

    return mu

def build_divisors_table(max_num):
    divisors = [[] for _ in range(max_num + 1)]

    for divisor in range(1, len(divisors)):
        for i in range(divisor, len(divisors), divisor):
            divisors[i].append(divisor)
    
    return divisors

In [73]:
class GaussFactorial:
    def __init__(self, max_num, modulus):
        self.modulus = modulus
        self.totient_table = build_totient_table(max_num)
        self.fact_div_results = build_fact_n_div_n_pow_n_mod_m_table(max_num, self.modulus)
        self.mobius_table = build_mobius_table(max_num)
        self.divisors_table = build_divisors_table(max_num)
    
    def __call__(self, num):
        n_pow_phi_n = pow(num, self.totient_table[num - 1], self.modulus)
        divisors_prod = 1
        for divisor in self.divisors_table[num]:
            current_value = pow(
                self.fact_div_results[num], int(self.mobius_table[num // divisor]), self.modulus
            )
            divisors_prod = (divisors_prod * current_value) % self.modulus
        print(n_pow_phi_n, self.totient_table[num - 1], divisors_prod)
        return (n_pow_phi_n * divisors_prod) % self.modulus
    
    def product_in_range(self, min_num, max_num):
        result = 1
        for num in range(min_num, max_num + 1):
            result = (result * self(num)) % self.modulus
        return result

for num in range(1, 5):
    print(f"g({num}) = {GaussFactorial(num, 10 ** 9 + 7)(num)}")

1 1 1
g(1) = 1
2 1 1
g(2) = 2
9 2 1
g(3) = 9
16 2 1
g(4) = 16


In [70]:
pow(3, 0, 5)

1

In [59]:
MAX_NUM = 10 ** 4
MOD = 10 ** 9 + 7

gauss_factorial = GaussFactorial(MAX_NUM, MOD)
gauss_factorial.product_in_range(1, MAX_NUM)

70986767

In [65]:
gauss_factorial.divisors_table[4]

[1, 2, 4]

In [66]:
for num in range(1, 10):
    print(GaussFactorial(num, MOD)(num))

1
4
500000044
666666842
41682943
600002337
320612901
903291660
306889692


In [48]:
divisors_table

[[],
 [1],
 [1, 2],
 [1, 3],
 [1, 2, 4],
 [1, 5],
 [1, 2, 3, 6],
 [1, 7],
 [1, 2, 4, 8],
 [1, 3, 9],
 [1, 2, 5, 10]]